## This notebook provides a framework for running meta-learning benchmark experiment for the different dataset.

In [ ]:
%load_ext autoreload
%autoreload 2

### Experiment config, hyper-parameters

In [ ]:
exp_config = {
    'exp_name':'benchmark-sin-easy', 'dataset': 'sin-easy', 'x_dim': 1, 'y_dim': 1
    #'exp_name':'benchmark-sin-hard', 'dataset': 'sin-hard', 'x_dim': 1, 'y_dim': 1
    #'exp_name':'benchmark-cauchy', 'dataset': 'cauchy', 'x_dim': 2, 'y_dim': 1
    #'exp_name':'benchmark-swissfel', 'dataset': 'swissfel', 'x_dim': 12, 'y_dim': 1
}

alpaca_model_config = {
    'nn_layers': [32, 32, 32], 'sigma_eps': 0.04, 'activation': 'tanh',
    'data_horizon': 5, 'test_horizon': 5, 'loss_type':'prior', 'impl_c': True,
    'fixL': True, 'loss_full_cov': True, 'learn_sigma': True
}

alpaca_opt_config = {
    'optimizer': 'AdamW', 'lr': 0.001, 'weight_decay': 5e-4, 'meta_batch_size': 5
}

retrain_alpaca = False
retrain_gpmeta = False

gpmeta_model_config = {
    'learning_mode': 'both', 'covar_module': 'NN', 'mean_module': 'NN',
    'feature_dim': 32, 'mean_nn_layers': (128, 128), 'kernel_nn_layers': (128, 128),
    'normalize_data': True,
}

gpmeta_opt_config = {
    'optimizer': 'Adam',
    'lr': 0.001, 'weight_decay': 0.3, 'lr_decay': 1.0,
    'meta_batch_size': 5
}

In [ ]:
# dataset preparation
import numpy as np
from lib.PACOH.experiments.data_sim import SinusoidDataset, CauchyDataset, provide_data

if "sin" in exp_config['dataset']:
    if exp_config['dataset'] == 'sin-easy':
        dataset = SinusoidDataset(random_state=41)
    elif exp_config['dataset'] == 'sin-hard':
        dataset = SinusoidDataset(amp_low=0.7, amp_high=1.4,
                         period_low=1.0, period_high=2.0,
                         x_shift_mean=0.0, x_shift_std=2.0,
                         y_shift_mean=5.0, y_shift_std=0.8,
                         slope_mean=0.5, slope_std=0.6,
                         noise_std=0.2, x_low=-5, x_high=5, random_state=41)
    
    data_train = dataset.generate_meta_train_data(n_tasks=20, n_samples=10)
    data_test, f_test = dataset.generate_meta_test_data(n_tasks=100, n_samples_context=10,
                                                        n_samples_test=100, return_f=True)
    
elif exp_config['dataset'] == 'cauchy':
    dataset = CauchyDataset(noise_std=0.05, ndim_x=2, random_state=31)
    data_train = dataset.generate_meta_train_data(n_tasks=20, n_samples=20)
    data_test = dataset.generate_meta_test_data(n_tasks=100,
                                            n_samples_context=20,
                                            n_samples_test=100)
    
elif exp_config['dataset'] == 'swissfel':
    data_train, _, data_test = provide_data(dataset='swissfel')

In [ ]:
import sys
import os
PACOH_DIR = os.path.dirname(os.path.abspath(''))+'/lib/PACOH'
sys.path.append(PACOH_DIR)

# import ALPaCa model
import tensorflow as tf
from lib.ALPaCA.main.alpaca import ALPaCA
from src.helper import alpaca_config, metalgp_config
from lib.PACOH.experiments.util import save_dict, dict_to_tabular_str

g1 = tf.Graph()
sess1 = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True), graph=g1)
alpaca = ALPaCA(alpaca_config(exp_config, alpaca_opt_config, alpaca_model_config), sess1, g1)
alpaca.construct_model()

# import PACOH model

from lib.PACOH.meta_learn.GPR_meta_mll import GPRegressionMetaLearned
import torch
import hashlib
import os

torch.set_num_threads(2)

gp_meta = GPRegressionMetaLearned(data_train, **metalgp_config(gpmeta_opt_config, gpmeta_model_config))

In [ ]:
# train models
alpaca_model_hash = hashlib.md5((str(exp_config)+str(alpaca_model_config)).encode('utf8')).hexdigest()
alpaca_model_path = os.path.dirname(os.path.abspath('')) + '/data/' + str(alpaca_model_hash) + '/'
os.makedirs(alpaca_model_path, exist_ok=True)
if not retrain_alpaca and os.path.isfile(alpaca_model_path + 'checkpoint'):
    alpaca.restore(alpaca_model_path + 'tf')
else:
    pass
    alpaca.train(data_train, 15000, valid_dataset=data_test)
    alpaca.save(alpaca_model_path + 'tf')
    save_dict({**exp_config, **alpaca_model_config, **alpaca_opt_config}, os.path.join(alpaca_model_path, 'config.json'))

In [ ]:
gpmeta_model_hash = hashlib.md5((str(exp_config)+str(gpmeta_model_config)).encode('utf8')).hexdigest()
gpmeta_model_path = os.path.dirname(os.path.abspath('')) + '/data/' + str(gpmeta_model_hash) + '/'
os.makedirs(gpmeta_model_path, exist_ok=True)
if not retrain_gpmeta and os.path.isfile(gpmeta_model_path + 'gpmeta.pth'):
    gp_meta.load_state_dict(torch.load(gpmeta_model_path + 'gpmeta.pth'))
else:
    gp_meta.meta_fit(valid_tuples=data_test[:100], log_period=1000, n_iter=10000)
    torch.save(gp_meta.state_dict(), gpmeta_model_path + 'gpmeta.pth')
    save_dict({**exp_config, **gpmeta_model_config, **gpmeta_opt_config}, os.path.join(gpmeta_model_path, 'config.json'))

### Reporting metrics including ll(log likelihood), RMSE (root mean square error), and calibration error on test set

In [ ]:
alpaca_ll, alpaca_rmse, alpaca_calib = alpaca.eval_datasets(data_test)
gpmeta_ll, gpmeta_rmse, gpmeta_calib = gp_meta.eval_datasets(data_test)
alpaca_results_dict = {
    'alpaca_ll': np.float64(alpaca_ll),
    'alpaca_rmse': np.float64(alpaca_rmse),
    'alpaca_calib': np.float64(alpaca_calib),
}
gpmeta_results_dict = {
    'gpmeta_ll': np.float64(gpmeta_ll),
    'gpmeta_rmse': np.float64(gpmeta_rmse),
    'gpmeta_calib': np.float64(gpmeta_calib)
}
print("\n"+
        "----------------------------------------\n" + \
        "                   Results              \n" + \
        "----------------------------------------" + \
        "%s" % dict_to_tabular_str({**alpaca_results_dict, **gpmeta_results_dict}) + \
        "----------------------------------------\n")

In [ ]:
from lib.PACOH.experiments.util import save_results
save_results(alpaca_results_dict, alpaca_model_path, log=True)
save_results(gpmeta_results_dict, gpmeta_model_path, log=True)

### Visualizations on per-time-step performance

In [ ]:
# eval models

if "sin" in exp_config['dataset']:
    test_idx = 2
    
    x_gt = np.linspace(-5., 5., 100)
    x_gt_tile = np.tile(x_gt, (100, 1)).reshape(100, 100, 1)
    y_gt = f_test[test_idx](x_gt)
    
    from src.benchmark_sinusoid import plot_sin_comparison, plot_sin_alpaca
    %matplotlib inline
    plot_sin_alpaca(alpaca, test_idx, data_test, x_gt_tile, x_gt, y_gt)
    plot_sin_comparison(alpaca, gp_meta, test_idx, data_test, x_gt, y_gt, alpaca_model_path)
